In [1]:
"""
MACE Stage Two: Time-Constrained Complementary Evolution

修改说明:
1. 类名从EoHS改为MACEEvolver
2. 整合CI算子作为独立模块
3. 为后续添加PI, SI, DI算子预留接口
"""

import numpy as np
from typing import Dict, List, Tuple, Any, Set
import os
import random
from src.run_hyper_heuristic.run_hyper_heuristic import run_hyper_heuristic
from src.run_hyper_heuristic.smoke_test import standalone_smoke_test
from src.run_hyper_heuristic.complementary_selection_simple import complementary_selection_milp


class MACEEvolver:
    """
    MACE Stage Two: Time-Constrained Complementary Evolution
    
    实现论文Algorithm 1的Stage Two部分:
    - 通过5个进化算子(CI, PI, SI, DI, EI)生成新启发式
    - 使用Complementary Selection选择最优组合
    - 支持多时间预算设置
    
    进化算子实现状态:
    - CI (Complementary Improvement)   ✅ 已实现
    - PI (Performance Improvement)     ✅ 已实现
    - SI (Specialization Improvement)  ✅ 已实现
    - DI (Diversity Improvement)       ⭐ 待实现
    - EI (Efficiency Improvement)      ✅ 已实现
    """
    
    def __init__(
        self,
        problem: str,
        api_key: str,
        heuristic_dir: str,
        task_description_file: str,
        output_dir: str,
        model: str,
        population_size: int = 5,
        max_evaluations: int = 100,
        time_limit: float = 600,
        llm_config_file: str = None,
        iterations_scale_factor: float = 1.0,
        result_dir: str = "result",
        milp_time_limit: float = 600.0
    ):
        """
        Args:
            problem: 问题类型 (tsp, jssp, cvrp, psp)
            api_key: LLM API密钥
            heuristic_dir: 启发式目录
            task_description_file: 任务描述文件
            output_dir: 输出目录
            model: LLM模型名称
            population_size: 种群大小 n (论文中default=10)
            max_evaluations: 最大评估次数 Nmax (论文中default=100)
            time_limit: 时间预算 Tmax (秒)
            llm_config_file: LLM配置文件
            iterations_scale_factor: 迭代缩放因子
            result_dir: 结果目录
            milp_time_limit: MILP求解时间限制(秒, 默认60)
        """
        self.problem = problem
        self.api_key = api_key
        self.heuristic_dir = heuristic_dir
        self.task_description_file = task_description_file
        self.output_dir = output_dir
        self.model = model
        self.n = population_size
        self.Nmax = max_evaluations
        self.time_limit = time_limit
        self.llm_config_file = llm_config_file
        self.iterations_scale_factor = iterations_scale_factor
        self.result_dir = result_dir
        self.milp_time_limit = milp_time_limit
        
        # 评估计数器
        self.evaluations = 0
        
        # 初始化进化算子
        from src.run_hyper_heuristic.ci_operator import CIOperator
        from src.run_hyper_heuristic.pi_operator import PIOperator
        from src.run_hyper_heuristic.si_operator import SIOperator
        from src.run_hyper_heuristic.di_operator import DIOperator
        from src.run_hyper_heuristic.ei_operator import EIOperator
        
        self.ci_operator = CIOperator(
            problem=problem,
            heuristic_dir=heuristic_dir,
            task_description_file=task_description_file,
            output_dir=output_dir,
            api_key=api_key,
            model=model
        )
        
        self.pi_operator = PIOperator(
            problem=problem,
            heuristic_dir=heuristic_dir,
            task_description_file=task_description_file,
            output_dir=output_dir,
            api_key=api_key,
            model=model
        )
        
        self.si_operator = SIOperator(
            problem=problem,
            heuristic_dir=heuristic_dir,
            task_description_file=task_description_file,
            output_dir=output_dir,
            test_data_dir="output/test_data",  # 测试数据目录
            api_key=api_key,
            model=model
        )

        self.di_operator = DIOperator(
        problem=problem,
        heuristic_dir=heuristic_dir,
        task_description_file=task_description_file,
        output_dir=output_dir,
        api_key=api_key,
        model=model
     )
        
        self.ei_operator = EIOperator(
            problem=problem,
            heuristic_dir=heuristic_dir,
            task_description_file=task_description_file,
            output_dir=output_dir,
            api_key=api_key,
            model=model
        )
        
        print(f"\n{'='*80}")
        print(f"🚀 MACE Evolver 初始化完成")
        print(f"{'='*80}")
        print(f"问题类型: {problem}")
        print(f"种群大小: {population_size}")
        print(f"最大评估次数: {max_evaluations}")
        print(f"时间预算: {time_limit}秒")
        print(f"互补选择: Gurobi MILP")
        print(f"MILP时间限制: {milp_time_limit}秒")
        print(f"{'='*80}\n")
    
    def initialize_population(self, results_dict: Dict) -> List[Dict]:
        """
        初始化种群 P0 (论文Algorithm 1, line 7)
        
        Args:
            results_dict: 初始启发式的评估结果
                格式: {heuristic_name: [score_1, score_2, ...]}
        
        Returns:
            population: 初始种群列表
        """
        population = []
        for heuristic_name, heuristic_scores in results_dict.items():
            heuristic_info = {
                'name': heuristic_name,
                'performance_vector': heuristic_scores,
                'avg_performance': np.mean(heuristic_scores)
            }
            population.append(heuristic_info)
        return population
    
    def evolute_population(
        self,
        population: List[Dict],
        results_dict: Dict,
        time_limit: float
    ) -> List[Dict]:
        """
        Evolute Population - 生成n个新启发式 (论文Algorithm 1, lines 10-15)
        
        当前实现的算子:
        - CI: Complementary Improvement (互补改进)
        - PI: Performance Improvement (性能改进)
        - SI: Specialization Improvement (专业化改进)
        
        待实现:
        - DI: Diversity Improvement (多样性改进)
        
        Args:
            population: 当前种群
            results_dict: 性能评估结果字典
            time_limit: 时间限制
        
        Returns:
            new_heuristics: 新生成的启发式列表
        """
        new_heuristics = []
        attempts = 0
        max_attempts = self.n * 5  # 允许一些失败尝试
        
        print(f"\n{'='*80}")
        print(f"🔄 开始Evolute Population，目标生成 {self.n} 个新启发式")
        print(f"{'='*80}")
        
        while len(new_heuristics) < self.n and attempts < max_attempts:
            attempts += 1
            
            # 随机选择算子 (论文Algorithm 2, line 1)
            # 当前实现了CI, PI, SI三个算子
            operator = random.choice(['CI', 'PI', 'SI', 'DI'])
            try:
                if operator == 'CI':
                    # CI算子: Complementary Improvement
                    print(f"\n  🔍 [{operator}] 尝试 #{len(new_heuristics)+1}/{self.n} (总尝试 {attempts}/{max_attempts})")
                    
                    # 将population转换为results_dict格式 (CI需要)
                    results_dict = {
                        h['name']: h['performance_vector']
                        for h in population
                    }
                    file_path, code = self.ci_operator.generate(results_dict)
                    
                elif operator == 'PI':
                    # PI算子: Performance Improvement
                    print(f"\n  🔧 [{operator}] 尝试 #{len(new_heuristics)+1}/{self.n} (总尝试 {attempts}/{max_attempts})")
                    
                    file_path, code = self.pi_operator.generate(population)
                
                elif operator == 'SI':
                    # SI算子: Specialization Improvement
                    print(f"\n  🎯 [{operator}] 尝试 #{len(new_heuristics)+1}/{self.n} (总尝试 {attempts}/{max_attempts})")
                    
                    file_path, code = self.si_operator.generate(population)
                
                # TODO: 后续添加DI算子
                elif operator == 'DI':
                    # DI算子: Deverisity Improvement
                    print(f"\n  🎯 [{operator}] 尝试 #{len(new_heuristics)+1}/{self.n} (总尝试 {attempts}/{max_attempts})")
                    file_path, code = self.di_operator.generate(population)
                
               
                if file_path and standalone_smoke_test(file_path, problem=self.problem, api_key=self.api_key):
                    # 评估新启发式
                    perf_vector, avg_perf = self.evaluate_heuristic(file_path, time_limit)
                    
                    # 检查可行性 (是否超时)
                    feasible = self._check_feasibility(perf_vector)
                    
                    if feasible:
                        # 直接加入新种群
                        print(f"  ✅ [{operator}] 成功: {os.path.basename(file_path)}")
                        new_heuristics.append({
                            'name': file_path,
                            'performance_vector': perf_vector,
                            'avg_performance': avg_perf
                        })
                        self.evaluations += 1
                    else:
                        # 需要效率改进 (EI算子)
                        print(f"  ⚠️  [{operator}] 超时, 应用EI算子")
                        file_path, code = self.ei_operator.generate(file_path)
                        
                        # 重新测试
                        if file_path and standalone_smoke_test(file_path, problem=self.problem, api_key=self.api_key):
                            perf_vector, avg_perf = self.evaluate_heuristic(file_path, time_limit)
                            feasible = self._check_feasibility(perf_vector)
                            
                            if feasible:
                                print(f"  ✅ [{operator}+EI] 成功: {os.path.basename(file_path)}")
                                new_heuristics.append({
                                    'name': file_path,
                                    'performance_vector': perf_vector,
                                    'avg_performance': avg_perf
                                })
                                self.evaluations += 1
                            else:
                                print(f"  ❌ [{operator}+EI] 仍然超时, 放弃")
                        else:
                            print(f"  ❌ [{operator}+EI] 烟雾测试失败")
                else:
                    print(f"  ❌ [{operator}] 烟雾测试失败")
                    
            except Exception as e:
                print(f"  ❌ [{operator}] 失败: {e}")
        
        print(f"\n{'='*80}")
        print(f"✓ Evolute Population 完成: 成功生成 {len(new_heuristics)}/{self.n} 个新启发式")
        print(f"{'='*80}\n")
        
        return new_heuristics
    
    def _check_feasibility(self, performance_vector: List[float]) -> bool:
        """
        检查启发式是否可行 (没有超时)
        
        Args:
            performance_vector: 性能向量
        
        Returns:
            bool: True表示可行, False表示超时
        """
        penalty_value = 1e10 - 1
        for score in performance_vector:
            if score > penalty_value:
                return False
        return True
    
    def evaluate_heuristic(
        self,
        heuristic_path: str,
        time_limit: float
    ) -> Tuple[List[float], float]:
        """
        评估单个启发式的性能
        
        Args:
            heuristic_path: 启发式文件路径
            time_limit: 时间限制
        
        Returns:
            (performance_vector, avg_performance): 性能向量和平均性能
        """
        results = run_hyper_heuristic(
            problem=self.problem,
            heuristic=heuristic_path,
            test_data="test_data",
            llm_config_file=self.llm_config_file,
            heuristic_dir=self.heuristic_dir,
            iterations_scale_factor=self.iterations_scale_factor,
            result_dir=self.result_dir,
            time_limit=time_limit
        )
        
        performance_vector = results
        avg_performance = np.mean(performance_vector)
        
        return performance_vector, avg_performance
    
    def complementary_population_management(
        self,
        current_pop: List[Dict],
        new_pop: List[Dict]
    ) -> List[Dict]:
        """
        互补种群管理 - 使用Gurobi MILP精确求解
        
        论文公式(9): MILP模型
        min η
        s.t. 选择n个启发式,最小化worst-case性能
        
        Args:
            current_pop: 当前种群 (n个)
            new_pop: 新生成的种群 (最多n个)
        
        Returns:
            selected: 选择的n个启发式
        """
        pool = current_pop + new_pop
        
        if len(pool) <= self.n:
            return pool
        
        # 使用MILP求解
        selected = complementary_selection_milp(
            pool=pool,
            n=self.n,
            time_limit=self.milp_time_limit,
            verbose=True
        )
        
        return selected
    
    def compute_cpi(self, population: List[Dict]) -> float:
        """
        计算互补性能指标 (CPI) - Complementary Performance Index
        
        论文公式(2): CPI = (1/m) * sum_i min_{h in H} f_i(h)
        
        Args:
            population: 当前种群
        
        Returns:
            cpi: 互补性能指标 (越小越好)
        """
        if not population:
            return float('inf')
        
        m = len(population[0]['performance_vector'])
        total = 0.0
        
        for j in range(m):
            # 每个实例上的最佳得分
            f_star = min(h['performance_vector'][j] for h in population)
            total += f_star
        
        return total / m
    
    def run(self, initial_results_dict: Dict, time_limit: float) -> List[Dict]:
        """
        主进化循环 (论文Algorithm 1, lines 7-21)
        
        Args:
            initial_results_dict: 初始启发式的评估结果
            time_limit: 时间限制
        
        Returns:
            population: 最终进化的启发式种群
        """
        print("\n" + "=" * 80)
        print("🚀 MACE Stage Two 开始进化")
        print("=" * 80)
        print(f"配置: 种群大小={self.n}, 最大评估次数={self.Nmax}")
        
        # 1. 初始化种群 P0 (Algorithm 1, line 7)
        population = self.initialize_population(initial_results_dict)
        print(f"\n✓ 初始化种群: {len(population)} 个启发式")
        initial_cpi = self.compute_cpi(population)
        print(f"✓ 初始CPI: {initial_cpi:.4f}")
        
        # 2. 进化循环 (Algorithm 1, lines 8-21)
        iteration = 0
        while self.evaluations < self.Nmax:
            iteration += 1
            print(f"\n{'='*80}")
            print(f"📈 第 {iteration} 代进化 (已评估: {self.evaluations}/{self.Nmax})")
            print(f"{'='*80}")
            
            # (a) Evolute Population: 生成n个新启发式 (lines 10-15)
            new_heuristics = self.evolute_population(population, initial_results_dict, time_limit)
            
            # 如果没有生成任何新启发式，提前终止
            if not new_heuristics:
                print("\n⚠️  未能生成新启发式，进化终止")
                break
            
            # (b) Complementary Selection: 从2n候选中选择n个 (line 17)
            population = self.complementary_population_management(population, new_heuristics)
            
            # 计算当前CPI
            current_cpi = self.compute_cpi(population)
            print(f"\n📊 当前种群 CPI: {current_cpi:.4f}")
            print(f"📈 相比初始CPI改进: {((initial_cpi - current_cpi) / initial_cpi * 100):.2f}%")
            
            # 检查是否达到最大评估次数
            if self.evaluations >= self.Nmax:
                print(f"\n✅ 达到最大评估次数 {self.Nmax}，进化结束")
                break
        
        print("\n" + "=" * 80)
        print("🎉 MACE Stage Two 进化完成!")
        print("=" * 80)
        
        return population

In [2]:
"""
MACE Stage Two 使用示例 - 整合CI算子

这个示例展示如何使用新的MACEEvolver类和CI算子
"""

import numpy as np
from typing import Dict, List
from src.run_hyper_heuristic.run_hyper_heuristic import evaluate_all_heuristics

def main():
    """主函数"""
    
    # ========================================================================
    # 配置参数
    # ========================================================================
    problem = "cvrp"  # 问题类型: tsp, jssp, cvrp, psp
    api_key = "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"
    model = "x-ai/grok-code-fast-1"
    
    # 目录配置
    heuristic_dir = "basic_heuristics"
    task_description_file = f"src/problems/{problem}/task_description.txt"
    output_dir = f"src/problems/{problem}/heuristics/basic_heuristics"
    test_data = "test_data"
    result_dir = "result"
    
    # MACE参数
    population_size = 10       # 种群大小 n
    max_evaluations = 3     # 最大评估次数 Nmax
    time_limit = 10 * 60      # 时间预算 Tmax (10分钟)
    iterations_scale_factor = 1.0
    
    
    # ========================================================================
    # 步骤 1: 评估初始启发式 (Stage One的输出)
    # ========================================================================
    print("=" * 80)
    print("📋 步骤 1: 评估初始启发式")
    print("=" * 80)
    
    initial_results_dict = evaluate_all_heuristics(
        problem=problem,
        heuristic_dir=heuristic_dir,
        test_data=test_data,
        iterations_scale_factor=iterations_scale_factor,
        result_dir=result_dir,
        save_to_file=True,
        time_limit=time_limit
    )
    
    print("\n初始评估结果:")
    for h_name, scores in initial_results_dict.items():
        avg = np.mean(scores)
        print(f"  {h_name}: avg={avg:.2f}, scores={scores}")
    
    
    # ========================================================================
    # 步骤 2: 运行 MACE Stage Two
    # ========================================================================
    print("\n" + "=" * 80)
    print("📋 步骤 2: 运行 MACE Stage Two")
    print("=" * 80)
    
    # 创建MACE进化器
    mace = MACEEvolver(
        problem=problem,
        api_key=api_key,
        heuristic_dir=heuristic_dir,
        task_description_file=task_description_file,
        output_dir=output_dir,
        model=model,
        population_size=population_size,
        max_evaluations=max_evaluations,
        time_limit=time_limit
    )
    
    # 运行进化
    final_population = mace.run(initial_results_dict, time_limit)

    # ========================================================================
    # 步骤 3: 输出最终结果
    # ========================================================================
    print("\n" + "=" * 80)
    print("📊 最终结果")
    print("=" * 80)
    
    print(f"🎯 最终启发式集大小: {len(final_population)}")
    print(f"🏆 最终 CPI: {mace.compute_cpi(final_population):.4f}")
    print(f"📈 总评估次数: {mace.evaluations}")
    
    print("\n📋 最终启发式集:")
    for i, h in enumerate(final_population, 1):
        print(f"  {i}. {h['name']}")
        print(f"     avg={h['avg_performance']:.2f}")
        print(f"     scores={h['performance_vector']}")


if __name__ == "__main__":
    # 运行完整流程
    main()

📋 步骤 1: 评估初始启发式
找到 3 个启发式算法:

test_data评估完成！结果汇总:

初始评估结果:
  cheapest_insertion_heuristic_c124: avg=21324.65, scores=[28461.686309123157, 17334.38901063604, 19094.113374214878, 15501.930150275148, 36724.6023804212, 14696.838954713738, 17211.643895105095, 21571.958822941902]
  clarke_wright_savings_heuristic_9bf6: avg=1250019658.92, scores=[29971.03542989956, 18947.85675532604, 18176.401094250046, 17888.510870077036, 34175.93393456769, 10000000000.0, 16925.93414384684, 21185.6841787287]
  nearest_neighbor_heuristic_30c1: avg=10000000000.00, scores=[10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0]

📋 步骤 2: 运行 MACE Stage Two

🚀 MACE Evolver 初始化完成
问题类型: cvrp
种群大小: 10
最大评估次数: 3
时间预算: 600秒
互补选择: Gurobi MILP
MILP时间限制: 600.0秒


🚀 MACE Stage Two 开始进化
配置: 种群大小=10, 最大评估次数=3

✓ 初始化种群: 3 个启发式
✓ 初始CPI: 20807.3497

📈 第 1 代进化 (已评估: 0/3)

🔄 开始Evolute Population，目标生成 10 个新启发式

  🎯 [SI] 尝试 #1/10 (总尝试 1/50)

🎯 [SI算子] Specialization Imp

D:\AppGallery\Anaconda\Lib\site-packages\sklearn\cluster\_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


✅ Smoke Test 通过!


D:\AppGallery\Anaconda\Lib\site-packages\sklearn\cluster\_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(
D:\AppGallery\Anaconda\Lib\site-packages\sklearn\cluster\_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(
D:\AppGallery\Anaconda\Lib\site-packages\sklearn\cluster\_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(
D:\AppGallery\Anaconda\Lib\site-packages\sklearn\cluster\_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(
D:\AppGallery\Anaconda\Lib\site-packages\sklearn\cluster\_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and wi

  ⚠️  [DI] 超时, 应用EI算子

⚡ [EI算子] Efficiency Improvement 开始
✓ 目标启发式: clustering_xyzw.py
  原因: 超时 (t(h') > T_max)
✓ 代码已保存到: src/problems/cvrp/heuristics/basic_heuristics\clustering_xyzw.py
✓ 代码已保存到: src/problems/cvrp/heuristics/basic_heuristics\clustering_xyzw.py

✅ [EI算子] 工作流完成!
📁 生成的优化启发式: src/problems/cvrp/heuristics/basic_heuristics\clustering_xyzw.py
📄 完整响应记录: src/problems/cvrp/heuristics/basic_heuristics\clustering_xyzw_full_response.txt
🔬 基于原始启发式: clustering_xyzw.py


🧪 运行Smoke Test: clustering_xyzw
📋 问题类型: cvrp
⚠️  检测到Markdown标记 (```)
正在清理代码...
✓ 代码已清理并保存
✅ Smoke Test 通过!


D:\AppGallery\Anaconda\Lib\site-packages\sklearn\cluster\_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(
D:\AppGallery\Anaconda\Lib\site-packages\sklearn\cluster\_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(
D:\AppGallery\Anaconda\Lib\site-packages\sklearn\cluster\_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(
D:\AppGallery\Anaconda\Lib\site-packages\sklearn\cluster\_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(
D:\AppGallery\Anaconda\Lib\site-packages\sklearn\cluster\_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and wi

  ❌ [DI+EI] 仍然超时, 放弃

  🔧 [PI] 尝试 #4/10 (总尝试 8/50)

🔧 [PI算子] Performance Improvement 开始
✓ 选择的父代启发式: cheapest_insertion_heuristic_c124
  排名方差 σ_r: 0.5000
✓ 代码已保存到: src/problems/cvrp/heuristics/basic_heuristics\improved_insertion_r7g9.py
✓ 代码已保存到: src/problems/cvrp/heuristics/basic_heuristics\improved_insertion_r7g9.py

✅ [PI算子] 工作流完成!
📁 生成的启发式代码: src/problems/cvrp/heuristics/basic_heuristics\improved_insertion_r7g9.py
📄 完整响应记录: src/problems/cvrp/heuristics/basic_heuristics\improved_insertion_r7g9_full_response.txt
🔬 基于父代: cheapest_insertion_heuristic_c124 (σ_r=0.5000)


🧪 运行Smoke Test: improved_insertion_r7g9
📋 问题类型: cvrp
✅ Smoke Test 通过!
  ✅ [PI] 成功: improved_insertion_r7g9.py

  🔧 [PI] 尝试 #5/10 (总尝试 9/50)

🔧 [PI算子] Performance Improvement 开始
✓ 选择的父代启发式: cheapest_insertion_heuristic_c124
  排名方差 σ_r: 0.5000
✓ 代码已保存到: src/problems/cvrp/heuristics/basic_heuristics\improved_insertion_b3k9.py
✓ 代码已保存到: src/problems/cvrp/heuristics/basic_heuristics\improved_insertion_b3k9.py

✅ [PI算子] 工作流完成!